# Computer Simulation Project
Simulating a discrete-event CPU scheduler.

**Authors**: <br>
Mohammadreza Mofayezi 98106059 <br>
Amirhossein Bagheri 98105621

## Import Libraries

In [323]:
import simpy
from typing import List

import pandas as pd
import numpy  as np
from queue import PriorityQueue

from scipy.stats import norm, expon, poisson

## Inputs
The parameters, number of processes and duration of simulation.

In [324]:
# parameters
X = 3
Y = 12
Z = 10
# number of processes
N = 100
# length of simulation
L = 800
# task threshold
k = 10
seed = 1234
T1 = 2
T2 = 4

In [325]:
np.random.seed(seed=seed)

## Job Creator

In [326]:
class Task:
    def __init__(self, id: int):
        self.id = id
        self.inter_arrival = int(poisson.rvs(X))
        self.service_time = int(expon.rvs(Y))
        self.remaining_service = self.service_time
        self.priority = self.get_priority()
        self.arrival_time = None
        self.finish = -1

    def get_priority(self):
        return np.random.choice([1, 2, 3], 1, p=[0.1, 0.2, 0.7])[0]

    def __eq__(self, __o: object) -> bool:
        return self.priority == __o.priority

    def __repr__(self) -> str:
        return f'Task {self.id} Priority: {self.priority} - InterArrival: {self.inter_arrival} - ServiceTime: {self.service_time} FinishTime: {self.finish} SystemTime: {self.finish-self.arrival_time} WatingTime: {self.finish-self.arrival_time-self.service_time}'

In [327]:
class JobCreator:
    def __init__(self, num_tasks: int):
        self.num_tasks = num_tasks
        self.tasks = PriorityQueue()
        self.all_tasks = []

    def run(self):
        start = 0
        task = Task(0)
        task.arrival_time = 0
        self.tasks.put((task.arrival_time,task.priority, -task.remaining_service, task))
        self.all_tasks.append(task)
        
        for i in range(1,self.num_tasks):
            task = Task(i)
            start += task.inter_arrival
            task.arrival_time = start
            self.tasks.put((task.arrival_time,task.priority, -task.remaining_service, task))
            self.all_tasks.append(task)

    def get_task(self,time: int):
        if self.tasks.queue[0][0] <= time:
            return self.tasks.get()[-1]
        return None
    
    def len(self):
        return len(self.tasks.queue)
    
    def empty_job(self):
        return self.tasks.empty()


In [328]:
class Que:
    def __init__(self):
        self.q = []
    
    def insert(self,task: Task):
        self.q.insert(0,task)
    
    def len(self):
        return len(self.q)
    
    def run(self,time: int):
        q = []
        for i in range(len(self.q)):
            if self.q[i].remaining_service > 0:
                q.append(self.q[i])
        self.q = q



    def get_task(self):
        return self.q[-1]
    
    def pop_task(self):
        return self.q.pop()

    def give_up(self, task: Task = None):
        return False

    def timesim(self):
        return float("inf")



class RR(Que):
    def __init__(self, T: int):
        super().__init__()
        self.T = T
    
    def give_up(self, task: Task = None):
        return task.remaining_service > self.T
    
    def timesim(self):
        return self.T

class Processor:
    def __init__(self, num_tasks: int, maxsimul: int, queues :List, P: List):
        self.Job_creator = JobCreator(num_tasks)
        self.Job_creator.run()
        self.queues = queues
        self.simultime = 0
        self.max_simultime = maxsimul
        self.K = 10
        self.order_of_finish = []
        self.P = P
    
    def job_loader(self, k: int):
        if sum([q.len() for q in self.queues]) >= k:
            return
        queue = PriorityQueue()
        for _ in range(k):
            if self.Job_creator.empty_job():
                break
            task = self.Job_creator.get_task(self.simultime)
            if task is None:
                break

            queue.put((task.priority,-task.remaining_service,task))
        
        for q in queue.queue:
            self.queues[0].insert(q[-1])
    
    def process(self, i: int):
        q = self.queues[i]
        task = q.get_task()
        give_up = q.give_up(task)
        time = min(q.timesim(),task.remaining_service)
        task.remaining_service = task.remaining_service - time
        self.simultime += time
        if task.remaining_service == 0:
            self.order_of_finish.append(task)
            task.finish = self.simultime
        q.run(self.simultime)
        if give_up :
            task = q.pop_task()
            self.queues[i+1].insert(task)
        self.job_loader(self.K)
    
    def queue_chooser(self):
        return np.random.choice(list(range(len(self.P))), 1, p=self.P)[0]
        


    def dispatcher(self):
        while (self.Job_creator.len() > 0 or sum([q.len() for q in self.queues]) > 0) and self.simultime <= self.max_simultime:
            if sum([q.len() for q in self.queues]) > 0:
                choose = self.queue_chooser()
                if self.queues[choose].len() > 0:
                    self.process(choose)

            else:
                print(self.simultime)
                self.job_loader(self.K)

    
    def run(self):
        self.job_loader(self.K)
        self.dispatcher()

        

processor = Processor(N, L, [RR(T1),RR(T2),Que()],[0.8,0.1,0.1])
processor.run()
print(f"number of jobs Done {len(processor.order_of_finish)}")
for i in processor.order_of_finish:
    print(i)


        

number of jobs Done 59
Task 0 Priority: 2 - InterArrival: 3 - ServiceTime: 13 FinishTime: 33 SystemTime: 33 WatingTime: 20
Task 1 Priority: 3 - InterArrival: 5 - ServiceTime: 13 FinishTime: 48 SystemTime: 43 WatingTime: 30
Task 2 Priority: 3 - InterArrival: 3 - ServiceTime: 13 FinishTime: 67 SystemTime: 59 WatingTime: 46
Task 3 Priority: 3 - InterArrival: 2 - ServiceTime: 12 FinishTime: 73 SystemTime: 63 WatingTime: 51
Task 4 Priority: 3 - InterArrival: 4 - ServiceTime: 14 FinishTime: 89 SystemTime: 75 WatingTime: 61
Task 5 Priority: 3 - InterArrival: 4 - ServiceTime: 14 FinishTime: 109 SystemTime: 91 WatingTime: 77
Task 6 Priority: 3 - InterArrival: 2 - ServiceTime: 12 FinishTime: 125 SystemTime: 105 WatingTime: 93
Task 7 Priority: 3 - InterArrival: 2 - ServiceTime: 12 FinishTime: 141 SystemTime: 119 WatingTime: 107
Task 8 Priority: 1 - InterArrival: 2 - ServiceTime: 12 FinishTime: 151 SystemTime: 127 WatingTime: 115
Task 10 Priority: 2 - InterArrival: 1 - ServiceTime: 16 FinishTime: 

## Scheduler

In [ ]:
class Scheduler:
    def __init__(self, env):
        self.env = env
        simpy.Resource(env, )

    def job_loader(self):
        pass

    def dispatcher(self):
        pass

1